In [1]:
from scipy.stats import uniform, norm, poisson
from math import exp, log
import numpy as np
import pandas as pd
import gc

# Part 2
- Assume the Merton jump model under the risk-neutral
measure
$dS_t = rS_{t−}dt + 𝜎S_{t−}dW_t + S_{t−}dJ_t$
where $W$ is a SBM, $r, 𝜎$ are constant parameters, and
$J_t = \sum_{j=1}^{N_t}(Y_j − 1)$ where $Y1, Y2, . . .$ are i.i.d. random variables and $N_t$ is a Poisson process with intensity parameter $𝜆$.
- Assume that $Y_i$ takes two values $y_− < y_+$ with probabilities
$p$ and $1 − p$
- Implement the program that takes as an input
$(S_0, r, 𝜎, y_−, y_+, p, 𝜆, N, n, T, K)$, where $T$ is the maturity
date, $N$ is the number of simulations, $n$ is the number of
observation dates, and produces the price estimate of the
Asian put option with the fixed strike $(K − \frac{1}{n} \sum_{i=1}^nS_{t_i})^+$ where $t_i = iT/n$.
- Select some parameter values and generate the table of
option prices for different pairs of $(N, m)$.

In [2]:
def simulate_Y(y_minus, y_plus, p, size):
  return np.where(np.random.uniform(size=size) < p, y_minus, y_plus)

def simulate_M(N, y_minus, y_plus, p):
  return np.sum(np.log(simulate_Y(y_minus, y_plus, p, N)))


def get_price(S0, r, sigma, y_minus, y_plus, p, lmbda, N, n, m, T, K):
  price = 0
  X0 = np.log(S0)
  dt = T/m
  X = np.zeros((m+1, N))
  X[0] = X0 * np.ones(N)
  for i in range(m):
    X[i+1] = X[i] + (r - 0.5*sigma**2) * dt + sigma * (dt**0.5) * np.random.normal(size=N) + np.array([simulate_M(np.random.poisson(lmbda * dt), y_minus, y_plus, p) for i in range(N)])
  S = np.exp(X)
  idx = np.arange(m/n, m+1, m/n, dtype=int)
  payoff = np.maximum(K - np.mean(S[idx], axis=0), 0)
  price = np.mean(payoff) * np.exp(-r * T)
  return price

In [3]:
S0 = 100         # initial price
r = 0.05         # model parameter r
sigma = 0.7      # model parameter σ
y_minus = 0.01   # modelparameter y-
y_plus = 1.1     # model parameter y+
p = 0.4          # model parameter p
lmbda = 1        # model parameter λ
N = 10000        # Number of simulations
n = 20           # number of observation dates
m = n * 5        # number of discretization steps, must be a multiple of n
T = 30           # maturity date
K = 120          # strike


In [4]:
df = pd.DataFrame({'N': [], 'm': [], 'price': []})
i = 0

In [5]:
# execution time 9 min
gc.collect()
for N in [10, 100, 1000, 10000, 100000]:
    for m in [n, n*2, n*5, n*10]:
        p = get_price(S0, r, sigma, y_minus, y_plus, p, lmbda, N, n, m, T, K)
        df.loc[i] = [N, m, p]
        i += 1

In [6]:
df

,N,m,price
0,10.0,20.0,24.741216
1,10.0,40.0,26.302240
2,10.0,100.0,26.448245
3,10.0,200.0,26.757812
4,100.0,20.0,26.411993
5,100.0,40.0,26.465333
6,100.0,100.0,26.394292
7,100.0,200.0,26.534559
8,1000.0,20.0,26.442813
9,1000.0,40.0,26.416119
